![](../assets/placeholder.png)

## Introduction
This notebook is designed to guide you through the process of building a simple chat application using Langchain and OpenAI's GPT-4 LLM. The focus is on demonstrating how to manage chat history for a seamless conversation experience with a stateless LLM.

### Chat
A lot of times we don't just want to ask a LLM one question. It actually is a session with some back and forth. To enable this chat experience, we need to keep track of previous messages with chat history. LLMs are stateless and can only deal with information within their context window. So we need to store this chat history separately and pass it to the LLM along with the new message so that it has all the context. This notebook demonstrates how to implement chat history management for maintaining context in conversations.

Let's start by setting up what hello world example we did in notebook one to highlight the need for chat history

In [1]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI
# Initialize the model with GPT-4 for our chat application
model = ChatOpenAI(model="gpt-4o")

In [3]:
from langchain_core.messages import HumanMessage, SystemMessage

system_message = "Respond back to the user greeting in Spanish"
human_message = "Hello AI! My name is GP"

messages = [
    SystemMessage(content=system_message),
    HumanMessage(content=human_message),
]

In [4]:
model.invoke(messages)

AIMessage(content='¡Hola GP! ¿Cómo estás?', response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 26, 'total_tokens': 34}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_319be4768e', 'finish_reason': 'stop', 'logprobs': None}, id='run-a549f421-580c-43eb-9bd4-f557e5f8a333-0', usage_metadata={'input_tokens': 26, 'output_tokens': 8, 'total_tokens': 34})

Now let's ask what our previous question was to see if it remembers

hint: it won't, as LLMs are stateless

In [5]:
human_message = "What is my name?"

In [6]:
model.invoke(human_message)

AIMessage(content="I'm sorry, but I don't have access to personal information about you, including your name. How can I assist you today?", response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 12, 'total_tokens': 37}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_319be4768e', 'finish_reason': 'stop', 'logprobs': None}, id='run-d826db5b-16eb-4e85-9b1d-31f054efb12d-0', usage_metadata={'input_tokens': 12, 'output_tokens': 25, 'total_tokens': 37})

In [7]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_session_history)
config = {"configurable": {"session_id": "1"}}

In [8]:
with_message_history.invoke(messages, config=config)

Parent run c99179dd-9059-4562-bf98-18d4028bc500 not found for run e46de6b2-0a10-49d4-afb2-59075820d81d. Treating as a root run.


AIMessage(content='¡Hola GP! ¿Cómo estás?', response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 26, 'total_tokens': 34}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_319be4768e', 'finish_reason': 'stop', 'logprobs': None}, id='run-e46de6b2-0a10-49d4-afb2-59075820d81d-0', usage_metadata={'input_tokens': 26, 'output_tokens': 8, 'total_tokens': 34})

In [9]:
human_message = "What is my name?"
with_message_history.invoke(human_message, config=config)

Parent run 53b39891-62b8-4e66-84a9-351383d9514c not found for run 6d365d42-bc7d-46a9-b2af-84263a9dfef7. Treating as a root run.


AIMessage(content='Tu nombre es GP. ¿En qué puedo ayudarte hoy?', response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 47, 'total_tokens': 59}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_319be4768e', 'finish_reason': 'stop', 'logprobs': None}, id='run-6d365d42-bc7d-46a9-b2af-84263a9dfef7-0', usage_metadata={'input_tokens': 47, 'output_tokens': 12, 'total_tokens': 59})